In [1]:
# pypower
from pypower.api import runpf,ppoption,printpf
from case33 import case33  
ppc = case33()  # 加载示例系统（4节点）
# 设置潮流计算选项
mpopt = ppoption(
    PF_ALG=2,         # 使用牛顿-拉夫逊法 (NR)
    VERBOSE=2,        # 输出详细信息
    OUT_ALL=0,        # 禁止详细的输出打印
    PF_V_CARTESIAN=1, # 使用直角坐标系计算电压
    PF_NR_MAX_IT=100, # NR法最大迭代次数
    PF_TOL=1e-5       # 潮流收敛容差
)

# 运行潮流计算
results, success = runpf(ppc, mpopt)

# 打印运行结果
if success:
    print("潮流计算成功！")
    
    # 打印总线电压
    print("\nBus Voltages (magnitude and angle):")
    print(results['bus'][:, [0, 7, 8]])  # bus number, voltage magnitude, angle

    # 打印发电机输出功率
    print("\nGenerator Power Outputs:")
    print(results['gen'][:, [0, 1, 2]])  # generator bus, P, Q

    # 打印支路功率流
    print("\nBranch Power Flows:")
    print(results['branch'][:, [0, 1, 13, 14]])  # from bus, to bus, P_from, Q_from
else:
    print("潮流计算未收敛！")

PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


iteration     max mismatch (p.u.)  
type   #        P            Q     
---- ----  -----------  -----------
  -    0    4.200e-02    6.000e-02
  <class 'type'>    1    6.979e-02    6.411e-02
  Q    1    1.360e-01    9.021e-03
  <class 'type'>    2    1.336e-01    3.821e-02
  Q    2    3.487e-02    5.205e-03
  <class 'type'>    3    2.961e-02    1.766e-02
  Q    3    9.216e-03    1.991e-03
  <class 'type'>    4    8.174e-03    2.985e-03
  Q    4    4.685e-03    9.801e-05
  <class 'type'>    5    3.835e-03    1.837e-03
  Q    5    2.211e-04    1.350e-04
  <class 'type'>    6    1.553e-04    1.784e-04
  Q    6    2.705e-04    1.253e-05
  <class 'type'>    7    2.193e-04    8.846e-05
  Q    7    4.568e-05    6.439e-06
  <class 'type'>    8    3.823e-05    2.311e-05
  Q    8    1.004e-05    1.644e-06
  <class 'type'>    9    7.993e-06    2.819e-06
Fast-decoupled power flow converged in 9 P-iterations and 8 Q-iterat

In [2]:
# 输出结果
if success:
    print("潮流计算成功！")
    print("潮流计算结果：")
    printpf(results)
else:
    print("潮流计算未收敛。")

潮流计算成功！
潮流计算结果：

Converged in 0.00 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             33     Total Gen Capacity      10.0         -10.0 to 10.0
Generators         1     On-line Capacity        10.0         -10.0 to 10.0
Committed Gens     1     Generation (actual)      3.0               2.0
Loads             32     Load                     3.7               2.3
  Fixed           32       Fixed                  3.7               2.3
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches          32     Losses (I^2 * Z)         0.20              0.14
Transformers       0     Branch Charging (inj)     -                0.0
Inter-ties         0     Total Inter-tie Flow     0.0           

Set parameter LicenseID to value 2764600
Set parameter MIPGap to value 1e-08
Set parameter OptimalityTol to value 1e-08
Set parameter FeasibilityTol to value 1e-08
Set parameter OutputFlag to value 1
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[arm] - Darwin 25.2.0 25C56)

CPU model: Apple M4 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
FeasibilityTol  1e-08
MIPGap  1e-08
OptimalityTol  1e-08

Optimize a model with 165 rows, 197 columns and 487 nonzeros (Min)
Model fingerprint: 0x1f0f46b3
Model has 32 linear objective coefficients
Model has 32 quadratic constraints
Coefficient statistics:
  Matrix range     [4e-05, 1e+00]
  QMatrix range    [4e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [8e-01, 2e+00]
  RHS range        [1e-03, 1e+00]
Presolve removed 69 rows and 69 columns
Presolve time: 0.00s
Presolved: 161 rows, 192 columns, 540 nonzeros
Presolved model has 32 second-order cone constraints

In [4]:
# ---------------------------提取并保存结果 --------------------------

if model.status == GRB.OPTIMAL:
    # 结果输出目录
    os.makedirs("output", exist_ok=True)

    # 对每个时段分别保存结果
    for t_idx in range(T):
        # --- ------------读取该时段负荷 (供后面输出 load 之用) ---
        ppc_t = case33(t_idx)
        P_d_t = list(-ppc_t["bus"][:,2] / ppc_t["baseMVA"])  # 负值在模型中表示消耗
        Q_d_t = list(-ppc_t["bus"][:,3] / ppc_t["baseMVA"])

        # ========== (A) Voltage results ==========
        #   "Bus", "Voltage Magnitude", "Voltage Angle (radians)"
        voltage_data = []
        for j in N:
            bus_j = j
            # 决策变量 v[t_idx,j] 是电压幅值平方
            Vmag = math.sqrt(v[t_idx, j].X)  # 开根号
            Vangle = 0.0                     # 二阶锥松弛中无相角，赋值为 0
            voltage_data.append([bus_j, Vmag, Vangle])
        voltage_df = pd.DataFrame(voltage_data, columns=["Bus", "Voltage Magnitude", "Voltage Angle (radians)"])
        voltage_df.to_csv(f"output/voltage_results_{t_idx}.csv", index=False)

        # ========== (B) Line flow results ==========
        #   "From Bus", "To Bus", "Active Power (MW)", "Reactive Power (MVAr)", "Current^2 (p.u.)"
        line_flow_data = []
        for (start_bus, end_bus, r_ij, x_ij) in E:
            valP = P[(t_idx, start_bus,end_bus,r_ij,x_ij)].X
            valQ = Q[(t_idx, start_bus,end_bus,r_ij,x_ij)].X
            valL = l[(t_idx,start_bus,end_bus,r_ij,x_ij)].X
            line_flow_data.append([start_bus, end_bus, valP, valQ, valL])
        line_flow_df = pd.DataFrame(
            line_flow_data,
            columns=["From Bus", "To Bus", "Active Power (pu)", "Reactive Power (pu)", "Current^2 (p.u.)"]
        )
        line_flow_df.to_csv(f"output/line_flow_results_{t_idx}.csv", index=False)

        # ========== (C) Node injection p,q ==========
        #   "Bus", "Active Injection (MW)", "Reactive Injection (MVAr)"
        node_inj_data = []
        for j in N:
            p_inj = p[t_idx,j].X
            q_inj = q[t_idx,j].X
            node_inj_data.append([j, p_inj, q_inj])
        node_inj_df = pd.DataFrame(
            node_inj_data,
            columns=["Bus", "Active Injection (pu)", "Reactive Injection (pu)"]
        )
        node_inj_df.to_csv(f"output/node_injection_results_{t_idx}.csv", index=False)

        # ========== (D) Generator output Pg,Qg ==========
        #   "Generator", "Bus", "Active Power (MW)", "Reactive Power (MVAr)"
        gen_output_data = []
        for g_idx in range(num_gen):
            bus_i = int(gen_data[g_idx, 0] - 1)
            Pg_val = P_g[t_idx, g_idx].X
            Qg_val = Q_g[t_idx, g_idx].X
            gen_output_data.append([g_idx, bus_i, Pg_val, Qg_val])
        gen_output_df = pd.DataFrame(
            gen_output_data,
            columns=["Generator", "Bus", "Active Power (pu)", "Reactive Power (pu)"]
        )
        gen_output_df.to_csv(f"output/gen_output_results_{t_idx}.csv", index=False)

        # ========== (E) Load demand (Pd,Qd) ==========
        #   "Bus", "Active Demand (MW)", "Reactive Demand (MVAr)"
        # 在模型里用负数表示负荷，所以这里恢复成正值输出
        load_data = []
        for j in N:
            # P_d_t[j], Q_d_t[j] 在模型里是负值
            load_data.append([j, -P_d_t[j], -Q_d_t[j]])
        load_df = pd.DataFrame(
            load_data,
            columns=["Bus", "Active Demand (MW)", "Reactive Demand (MVAr)"]
        )
        load_df.to_csv(f"output/fixed_load_demand_{t_idx}.csv", index=False)
        # ========== (F) DR Load demand (Pdr,Qdr) ==========
        DR_output_data = []
        for dr_idx in range(num_dr):
            bus_i = int(DRload_data[dr_idx, 0] - 1)
            Pdr_val = P_DR[t_idx, dr_idx].X
            DR_output_data.append([dr_idx, bus_i, Pdr_val])
            DR_output_df = pd.DataFrame(
            DR_output_data,
            columns=["DR", "Bus", "Active Power (MW)"] # 正数表示削减，负数表示负荷升高
        )
        # DR_output_df.to_csv(f"output/DR_output_results_{t_idx}.csv", index=False)
    print("Optimal solution found and results saved in 'output' folder.")
else:
    print(f"Model is not optimal or infeasible. Status code = {model.status}")

Optimal solution found and results saved in 'output' folder.


In [5]:
import AngleRecoveryCondition as ARC # 相角恢复条件
result,beta_t,A_t = ARC.compute_Bf_beta(ppc_0, E, P, Q, v, t_idx)
print("结果:", result)

ppc可能是一个辐射网络，因此没有连支，结果为空
结果: []


/Users/taylorchen/Documents/CETLAB/规划中心/配网约束/AngleRecoveryCondition.py:111: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  beta_combined,beta_t,beta_l = np.array(compute_beta(E, P, Q, v, t_idx, tree_branch_indices))


In [6]:
import cmath
import numpy as np

# 计算公式中的变量
def compute_S_I_V_s(t_idx, N, E, v, P, Q, l, p, q, A_t, beta_t):
    # 计算 cita = inv(A_t) @ beta_t
    cita = np.linalg.inv(A_t).T @ beta_t # 节点-支路关联矩阵按国内教材惯例，要取转置

    # 初始化结果字典
    S = {}
    I = {}
    V = {}
    s_0 = {}

    # 计算 S_{ij} 和 I_{ij} 的值
    for (start_bus, end_bus, r_ij, x_ij) in E:
        valP = P[(t_idx, start_bus, end_bus, r_ij, x_ij)].X
        valQ = Q[(t_idx, start_bus, end_bus, r_ij, x_ij)].X
        valL = l[(t_idx, start_bus, end_bus, r_ij, x_ij)].X

        # 计算 S_{ij}
        S_ij = valP + 1j * valQ

        # 修正 theta_i 索引（去除平衡节点影响）
        theta_i = cita[start_bus - 1]  # 修正索引
        angle_S_ij = cmath.phase(S_ij)

        # 计算 I_{ij}
        I_ij = np.sqrt(valL) * cmath.exp(1j * (theta_i*np.pi/180 - angle_S_ij))
        I[(start_bus, end_bus)] = I_ij

        # 存储 S_ij
        S[(start_bus, end_bus)] = S_ij

    # 计算 V_i 和 s_0
    for j in N:
        v_i = v[t_idx, j].X  # 获取电压幅值
        if j == 0:
            continue  # 跳过平衡节点
        theta_j = cita[j - 1]  # 修正索引
        # 计算电压相量 V_i
        V_i = np.sqrt(v_i) * cmath.exp(1j * theta_j*np.pi/180)
        V[j] = V_i

        # 计算 s_0
        p_0 = p[t_idx, j].X
        q_0 = q[t_idx, j].X
        s_0[j] = p_0 + 1j * q_0

    return S, I, V, s_0

# 调用计算函数
S, I, V, s_0 = compute_S_I_V_s(t_idx, N, E, v, P, Q, l, p, q, A_t, beta_t)

# 输出结果
print("S_{ij}:")
for key, value in S.items():
    print(f"({key[0]}, {key[1]}): {value}")

print("\nI_{ij} (幅值, 相角 [°]):")
for key, value in I.items():
    magnitude = abs(value)  # 计算幅值
    angle = cmath.phase(value) * 180 / np.pi  # 计算相角（转换为度）
    print(f"({key[0]}, {key[1]}): ({magnitude:.6f}, {angle:.4f}°)")

print("\nV_i (幅值, 相角 [°]):")
for key, value in V.items():
    magnitude = abs(value)  # 计算幅值
    angle = cmath.phase(value) * 180 / np.pi  # 计算相角（转换为度）
    print(f"{key+1}: ({magnitude:.6f}, {angle:.4f}°)") # 改变输出索引，和 pypower对应


print("\ns_0:")
for key, value in s_0.items():
    print(f"{key+1}: {value}")


S_{ij}:
(0, 1): (0.39176773630544376+0.2435141195700008j)
(1, 2): (0.3444299295663858+0.22078225162759665j)
(2, 3): (0.23628952846749185+0.16842005971246332j)
(3, 4): (0.22229948020933543+0.15940654879693708j)
(4, 5): (0.2144295856356866+0.15545418318508067j)
(5, 6): (0.10952674304682085+0.05278877801147967j)
(6, 7): (0.08933529100488019+0.04215592265466074j)
(7, 8): (0.068851493073169+0.031996039475429776j)
(8, 9): (0.062433438879828676+0.0296956898617923j)
(9, 10): (0.05607734723451193+0.027443287736461482j)
(10, 11): (0.05152197698551284+0.02442498119407539j)
(11, 12): (0.045433863456551156+0.020895845364420546j)
(12, 13): (0.03916723973631999+0.01718606988722078j)
(13, 14): (0.027094323488659186+0.009090091326595929j)
(14, 15): (0.02105862601549761+0.008058319970492306j)
(15, 16): (0.015030479092667878+0.006037765137496243j)
(16, 17): (0.009005315561495912+0.004004168215086507j)
(1, 18): (0.03611376387759495+0.01610789815020644j)
(18, 19): (0.02709766839370299+0.012092538735439133j